In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from umap import UMAP
from hdbscan import HDBSCAN
import hdbscan
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import PartOfSpeech
import os
import json
import openml
import spacy

/Users/ivang/miniconda3/envs/mvp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

# Data preparation

Download datasets

In [4]:
# datasets_list = openml.datasets.list_datasets()
# ids = list(datasets_list.keys())
# print("Number of datasets: ", len(ids))

# datasets = []
# # # Removing non-dataset entries 
# # ids.remove(4537)
# # ids.remove(4546)
# # ids.remove(4562)

# for dataset_id in tqdm(ids, desc="Downloading datasets"):
#     try:
#         dataset = openml.datasets.get_dataset(dataset_id, download_features_meta_data=True)
#     except openml.exceptions.OpenMLServerException:
#         # If the first attempt fails, try again without features metadata
#         dataset = openml.datasets.get_dataset(dataset_id, download_features_meta_data=False)
#     datasets.append(dataset)

# datasets[0]

In [5]:
# df = pd.DataFrame([dataset.__dict__ for dataset in datasets])
# # Add the OpenML URL to the dataframe
# df['openml_url'] = [dataset.openml_url for dataset in datasets]
# df['upload_date'] = [dataset.upload_date for dataset in datasets]
# # All datasets which have None description, make it an empty string ""
# df['description'] = df['description'].fillna('')
# df

Save dataframe, so datasets aren't refetched every time.

In [6]:
# df.to_pickle("datasets_checkpoint.pkl")

Load datasets from the saved CSV

In [7]:
df = pd.read_pickle("datasets_checkpoint.pkl")
df

,ignore_attribute,dataset_id,name,version,description,cache_format,format,creator,contributor,collection_date,...,parquet_file,_dataset,_parquet_url,_features,_qualities,_no_qualities_found,data_pickle_file,data_feather_file,feather_attribute_file,openml_url
0,None,2,anneal,1,**Author**: Unknown. Donated by David Sterling...,pickle,ARFF,"[David Sterling, Wray Buntine]",David Sterling and Wray Buntine,1990,...,None,None,http://openml1.win.tue.nl/dataset2/dataset_2.pq,"{0: [0 - family (nominal)], 1: [1 - product-ty...",None,False,None,None,None,https://www.openml.org/d/2
1,None,3,kr-vs-kp,1,Author: Alen Shapiro\nSource: [UCI](https://ar...,pickle,ARFF,Alen Shapiro,Rob Holte,1989-08-01,...,None,None,http://openml1.win.tue.nl/dataset3/dataset_3.pq,"{0: [0 - bkblk (nominal)], 1: [1 - bknwy (nomi...",None,False,None,None,None,https://www.openml.org/d/3
2,None,4,labor,1,**Author**: Unknown\n**Source**: Collective Ba...,pickle,ARFF,Collective Bargaining Review of Labour Canada,Stan Matwin,1988-11-01,...,None,None,http://openml1.win.tue.nl/dataset4/dataset_4.pq,"{0: [0 - duration (numeric)], 1: [1 - wage-inc...",None,False,None,None,None,https://www.openml.org/d/4
3,None,5,arrhythmia,1,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",pickle,ARFF,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",None,1998-01-01,...,None,None,http://openml1.win.tue.nl/dataset5/dataset_5.pq,"{0: [0 - age (numeric)], 1: [1 - sex (nominal)...",None,False,None,None,None,https://www.openml.org/d/5
4,None,6,letter,1,**Author**: David J. Slate \n**Source**: [UCI...,pickle,ARFF,David J. Slate,None,1991-01-01,...,None,None,http://openml1.win.tue.nl/dataset6/dataset_6.pq,"{0: [0 - x-box (numeric)], 1: [1 - y-box (nume...",None,False,None,None,None,https://www.openml.org/d/6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,None,46514,US_Stocks_Financial_Indicators_2018,5,# 200+ Financial Indicators of US Stocks (2018...,pickle,arff,Nicolas Carbone,Yayun Li,2024-11-23,...,None,None,https://openml1.win.tue.nl/datasets/0004/46514...,"{0: [0 - revenue (numeric)], 1: [1 - revenue_g...",None,False,None,None,None,https://www.openml.org/d/46514
5846,None,46515,Financial-Risk-Assessment-Preprocessed,1,Dataset is uploaded from kaggle. https://www.k...,pickle,arff,PREETHAM GOUDA,None,None,...,None,None,https://openml1.win.tue.nl/datasets/0004/46515...,"{0: [0 - age (numeric)], 1: [1 - education_lev...",None,False,None,None,None,https://www.openml.org/d/46515
5847,None,46516,Loan_Status_Classification,12,# Loan Approval Classification Dataset\n\nThis...,pickle,arff,Ta-wei Lo,Yayun Li,2024-11-22,...,None,None,https://openml1.win.tue.nl/datasets/0004/46516...,"{0: [0 - person_age (numeric)], 1: [1 - person...",None,False,None,None,None,https://www.openml.org/d/46516
5848,None,46517,HMEQ_Data_Preprocessed,1,Predict clients who default on their loan. Dat...,pickle,arff,A. Vallala,None,None,...,None,None,https://openml1.win.tue.nl/datasets/0004/46517...,"{0: [0 - loan (numeric)], 1: [1 - mortdue (num...",None,False,None,None,None,https://www.openml.org/d/46517


In [ ]:
# from augmenters import (
#     IdAugmenter,
#     TagAugmenter,
#     FeatureAugmenter, 
#     DatasetAugmenter,
#     SimilarityAugmenter,
#     NameAugmenter,
#     ScrapyAugmenter
# )

# description_column = 'description'
# dataset_id_column = 'dataset_id'
# tag_column = 'tag'
# features_column = '_features'
# name_column = 'name'
augmented_column = 'augmented_description'
# similar_datasets_column = 'similar_datasets'
# scraped_column = 'scraped_data'
# prompt_description_column = 'prompt_description'

# augmenters = [
#     TagAugmenter(description_column, tag_column, augmented_column, dataset_id_column, json_file_path="./GPT_semantic_tags.json"),
#     NameAugmenter(description_column, name_column, augmented_column),
#     FeatureAugmenter(description_column, features_column, augmented_column, max_features=100, reduce_features=True),
#     ScrapyAugmenter(description_column, scraped_column, augmented_column),
#     SimilarityAugmenter(description_column, augmented_column, similar_datasets_column),
#     IdAugmenter(description_column, dataset_id_column, augmented_column),
# ]

# dataset_augmenter = DatasetAugmenter(augmenters=augmenters)
# augmented_df = dataset_augmenter.augment(df.copy())
# augmented_df

,ignore_attribute,dataset_id,name,version,description,cache_format,format,creator,contributor,collection_date,...,_features,_qualities,_no_qualities_found,data_pickle_file,data_feather_file,feather_attribute_file,openml_url,augmented_description,scraped_data,similar_datasets
0,None,2,anneal,1,**Author**: Unknown. Donated by David Sterling...,pickle,ARFF,"[David Sterling, Wray Buntine]",David Sterling and Wray Buntine,1990,...,"{0: [0 - family (nominal)], 1: [1 - product-ty...",None,False,None,None,None,https://www.openml.org/d/2,"ID: 2\n\nName: anneal\n\nTags: study_1, study_...",Dataset Information Additional Information Att...,[]
1,None,3,kr-vs-kp,1,Author: Alen Shapiro\nSource: [UCI](https://ar...,pickle,ARFF,Alen Shapiro,Rob Holte,1989-08-01,...,"{0: [0 - bkblk (nominal)], 1: [1 - bknwy (nomi...",None,False,None,None,None,https://www.openml.org/d/3,ID: 3\n\nName: kr-vs-kp\n\nTags: mythbusting_1...,Dataset Information Additional Information The...,[]
2,None,4,labor,1,**Author**: Unknown\n**Source**: Collective Ba...,pickle,ARFF,Collective Bargaining Review of Labour Canada,Stan Matwin,1988-11-01,...,"{0: [0 - duration (numeric)], 1: [1 - wage-inc...",None,False,None,None,None,https://www.openml.org/d/4,"ID: 4\n\nName: labor\n\nTags: mythbusting_1, s...",Dataset Information Additional Information Dat...,[]
3,None,5,arrhythmia,1,"**Author**: H. Altay Guvenir, Burak Acar, Hald...",pickle,ARFF,"[H. Altay Guvenir, Burak Acar, Haldun Muderris...",None,1998-01-01,...,"{0: [0 - age (numeric)], 1: [1 - sex (nominal)...",None,False,None,None,None,https://www.openml.org/d/5,"ID: 5\n\nName: arrhythmia\n\nTags: sport, stud...",Dataset Information Additional Information Thi...,[]
4,None,6,letter,1,**Author**: David J. Slate \n**Source**: [UCI...,pickle,ARFF,David J. Slate,None,1991-01-01,...,"{0: [0 - x-box (numeric)], 1: [1 - y-box (nume...",None,False,None,None,None,https://www.openml.org/d/6,"ID: 6\n\nName: letter\n\nTags: AzurePilot, Azu...",Dataset Information Additional Information The...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,None,46514,US_Stocks_Financial_Indicators_2018,5,# 200+ Financial Indicators of US Stocks (2018...,pickle,arff,Nicolas Carbone,Yayun Li,2024-11-23,...,"{0: [0 - revenue (numeric)], 1: [1 - revenue_g...",None,False,None,None,None,https://www.openml.org/d/46514,ID: 46514\n\nName: US_Stocks_Financial_Indicat...,,[]
5846,None,46515,Financial-Risk-Assessment-Preprocessed,1,Dataset is uploaded from kaggle. https://www.k...,pickle,arff,PREETHAM GOUDA,None,None,...,"{0: [0 - age (numeric)], 1: [1 - education_lev...",None,False,None,None,None,https://www.openml.org/d/46515,ID: 46515\n\nName: Financial-Risk-Assessment-P...,,[]
5847,None,46516,Loan_Status_Classification,12,# Loan Approval Classification Dataset\n\nThis...,pickle,arff,Ta-wei Lo,Yayun Li,2024-11-22,...,"{0: [0 - person_age (numeric)], 1: [1 - person...",None,False,None,None,None,https://www.openml.org/d/46516,ID: 46516\n\nName: Loan_Status_Classification\...,,[]
5848,None,46517,HMEQ_Data_Preprocessed,1,Predict clients who default on their loan. Dat...,pickle,arff,A. Vallala,None,None,...,"{0: [0 - loan (numeric)], 1: [1 - mortdue (num...",None,False,None,None,None,https://www.openml.org/d/46517,ID: 46517\n\nName: HMEQ_Data_Preprocessed\n\nT...,,[]


### Save augmented descriptions

In [ ]:
# augmented_df.to_csv("augmented_df_checkpoint.csv", index=False)
# data = augmented_df[augmented_column].tolist()
augmented_df = pd.read_csv("augmented_df_checkpoint.csv")
data = pd.read_csv("augmented_df_checkpoint.csv")[augmented_column].tolist()
len(data)

5354

# Calculate embeddings

In [10]:
# embedding_model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")

In [11]:
# embeddings = embedding_model.encode(data, show_progress_bar=True, batch_size=1)
# np.save('embeddings_checkpoint.npy', embeddings)

In [12]:
# load embeddings
embeddings = np.load('embeddings_checkpoint.npy')
len(embeddings)

5354

In [13]:
# print(len(data)-len(embeddings))
# calculate new embeddings in case new data comes in
# new_embeddings = embedding_model.encode(data[len(embeddings):], show_progress_bar=True, batch_size=1)
# compare the new embeddings with the old ones
# print(len(embeddings))
# print(len(new_embeddings))
# extend the embeddings
# embeddings = np.concatenate([embeddings, new_embeddings])
# print(len(embeddings))
# np.save('embeddings_checkpoint.npy', embeddings)

# Train topic model

In [ ]:
# # Only run once
# vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 1))

# try:
#     representation_model = PartOfSpeech(model="en_core_web_lg")
#     print("Model already installed")
# except OSError:
#     # If model isn't installed, download it
#     print("Model not found, downloading...")
#     import os
#     os.system("python -m spacy download en_core_web_lg")


# umap_model = UMAP(
#     n_neighbors=3,
#     n_components=5,
#     min_dist=0.008,
#     random_state=42,
# )

# hdbscan_model = HDBSCAN(
#     min_cluster_size=3,
#     metric="euclidean",
#     cluster_selection_method="eom",
#     prediction_data=True,
# )

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# model = BERTopic(
#     verbose=True,
#     min_topic_size=2,
#     ctfidf_model=ctfidf_model,
#     calculate_probabilities=True,
#     umap_model=umap_model,
#     # embedding_model=embedding_model, # don't pass embedding model, just pass embeddings in the next step (pre-computed)
#     vectorizer_model=vectorizer_model,
#     representation_model=representation_model,
# )

# topics, probs = model.fit_transform(data, embeddings)
# topics_reduced = model.reduce_outliers(data, topics, probabilities=probs, strategy="probabilities")
# model.update_topics(data, topics=topics_reduced)
# np.save('topics_checkpoint.npy', topics)
# np.save('topics_reduced_checkpoint.npy', topics_reduced)
# np.save('probs_checkpoint.npy', probs)
# model.save("model", serialization="safetensors", save_ctfidf=True)

2024-12-07 14:28:27,541 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [ ]:
model = BERTopic.load("model")
topics = np.load('topics_checkpoint.npy')
topics_reduced = np.load('topics_reduced_checkpoint.npy')
probs = np.load('probs_checkpoint.npy')

2024-12-07 12:34:00,735 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


In [56]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,28,0_binarized_converts_classifying_binaryclass,"[binarized, converts, classifying, binaryclass...","[ID: 778\n\nName: bodyfat\n\nTags: binarized, ..."
1,1,18,1_study16_bng_study69_artificial,"[study16, bng, study69, artificial, bngionosph...",[ID: 146\n\nName: BNG(ionosphere)\n\nTags: art...
2,2,18,2_chemokine_cc_chemotaxis_cxc,"[chemokine, cc, chemotaxis, cxc, motif, go0016...",[ID: 4076\n\nName: QSAR-TID-10773\n\nTags: MTL...
3,3,17,3_sepallength_petalwidth_petallength_sepalwidth,"[sepallength, petalwidth, petallength, sepalwi...",[ID: 43387\n\nName: Oranges-vs.-Grapefruit\n\n...
4,4,16,4_cryptocurrency_cryptocurrencies_bitcoin_usd,"[cryptocurrency, cryptocurrencies, bitcoin, us...",[ID: 43391\n\nName: Ethereum-Cryptocurrency-Hi...
...,...,...,...,...,...
941,941,4,941_kinaselike_cyclindependent_q8ivw4_q92772,"[kinaselike, cyclindependent, q8ivw4, q92772, ...",[ID: 3171\n\nName: QSAR-TID-101309\n\nTags: MT...
942,942,3,942_delta_pkc_q05655_p05771,"[delta, pkc, q05655, p05771, p28867, ceramide,...",[ID: 4080\n\nName: QSAR-TID-11636\n\nTags: MTL...
943,943,5,943_titanic_embarked_pclass_fare,"[titanic, embarked, pclass, fare, survived, su...",[ID: 40704\n\nName: Titanic\n\nTags: derived\n...
944,944,4,944_cifar10_svhn_images_unsupervised,"[cifar10, svhn, images, unsupervised, labeled,...",[ID: 41103\n\nName: STL-10\n\nTags: AzurePilot...


In [367]:
prompt = '''Based on the below information, extract and synthesize human-readable tags/keywords/themes from the text, capitalized first letters of words. What is the main human-readable theme or subject matter discussed in the provided texts? What is the overarching, high-level theme of the texts, e.g. "Music", "Sports", "Environment", etc.? Please provide overarching themes that tie the different pieces of information together. What is/are the overarching, highest level theme(s) that you could use as a keyword(s)? Prefer single word tags/keywords, e.g. "Tennis" rather than "Tennis Match", "Prison" rather than "Prison Time", etc., however, if the term makes sense only as a whole, e.g. "Republic of the Congo", "COVID-19", then use it! Consider adding synonyms as well, e.g. for "Influenza", add "Flu", for "Car", add "Automobile", etc.! Some examples of human-readable themes are   "Agriculture", "Astronomy", "Chemistry", "Computational Universe", "Computer Systems", "Climate and Environment", "Culture", "Demographics", "Earth Science", "Economics", "Education", "Engineering", "Finance", "Geography", "Government", "Games", "Health", "History", "Human Activities", "Images", "Language", "Law", "Life Science", "Machine Learning", "Manufacturing", "Mathematics", "Medicine", "Meteorology", "Physical Sciences", "Politics", "Social Media", "Sociology", "Statistics", "Text & Literature",  "Transportation". Avoid tags/keywords that are too specific, e.g. "Serine Threonine Protein Kinase". Good theme examples are: "Birds", "Species Migration", "Air Pollution", or "War", "Government", "International Relations", "Politics". And be concise in theme generation, e.g. instead of "Income Prediction", say "Income", instead of "Demographic Information", say "Demographics"! Another important rule to obey - place more focus on the dataset names for theme extraction, if they exist. Say {"topic": {"themes": ["example1", "example2", ...], "overarching_themes": ["example13", "...", ...]}, "texts": [{"example1": 0.93, "example2": 0.68, "...": ..., ..., "example13": ..., ... (strings corresponding to themes and overarching themes for text 1, all of them)}], {"example1": 0.87, "example2": 0.34, "...": ..., ..., "example13": ..., ... (strings corresponding to themes and overarching themes for text 2, all of them)}]} and give your answer in JSON format, where under "topic" you put all themes and overarching themes, and in "texts", you give a confidence score in each theme and overarching theme for each text. "themes" and "overarching_themes" shouldn't overlap. If a theme is overarching and common to all texts, it should be present in all texts with a high score. Give only the ones with highest scores.
For example, for this text:
ID: 506

Name: Biden Administration

The Biden administration is preparing to roll out a sweeping border executive action as early as Tuesday, according to two sources familiar with the discussions, who cautioned that timing is fluid.

White House officials have begun reaching out to mayors who represent cities along the US southern border to potentially join President Joe Biden when he announces the order, two other sources familiar with those conversations said.

For weeks, administration officials have been working through an executive action that would dramatically limit migrants’ ability to seek asylum at the US southern border — part of a strategy to try to give Biden the upper hand on one of his Republican rival’s key campaign issues. The action is designed to potentially blunt Republican attacks on border security and preempt former President Donald Trump ahead of the first presidential debate, which will be held on June 27 on CNN.
---
ID: 401

Name: Name: Trump conviction

Now that a New York jury has convicted former President Donald Trump of all 34 felony charges of falsifying business records, the next obvious question is: Can a convicted felon run for president?

Definitely.

Trump meets all three requirements. There is, arguably, another criterion laid out in the 14th Amendment, where it states that no one who has previously taken an oath of office who engages in insurrection can be an officer of the US. But the US Supreme Court ruled earlier this year that Congress would have to pass a special law invoking this prohibition. That’s not happening any time soon.

Judge Juan Merchan has scheduled Trump’s sentencing for July 11, which happens to be four days before the start of the Republican National Convention that is scheduled to take place in Milwaukee.

It is technically possible, although perhaps unlikely for a first-time offender, that Trump could be sentenced to prison time.
---
ID: 51376

Name: Trump has vowed to give green cards to college grads. Could that actually happen?

The candidate known for touting immigration crackdowns told a group of tech investors that he wanted to help foreign students stay in the US.

“What I want to do, and what I will do, is — you graduate from a college, I think you should get automatically, as part of your diploma, a green card to be able to stay in this country,” Trump said during a June interview with “The All-In Podcast.”

If the president-elect pursues this proposal after he takes office, and if Congress passes the legislation that would be required to enact it, the policy could pave the way for potentially millions of international students to become legal permanent residents.
---
This would be your answer:
{
  "topic": {
    "themes": [
      "Biden Administration",
      "Border",
      "Executive Action",
      "Asylum",
      "Immigration",
      "Trump",
      "Felony",
      "Business Records",
      "Presidential Campaign",
      "Republican",
      "Debate",
      "Former President",
      "Conviction",
      "Sentencing",
      "Prison",
      "14th Amendment",
      "Insurrection",
      "Supreme Court",
      "Republican National Convention",
      "College",
      "Green Card",
      "Legislation",
      "Student"
    ],
    "overarching_themes": [
      "Politics",
      "Government",
      "Law",
      "Justice",
      "Elections",
      "Education"
    ]
  },
  "texts": { 
    506: {
      "Biden Administration": 0.96,
      "Border": 0.92,
      "Executive Action": 0.91,
      "Asylum": 0.88,
      "Immigration": 0.84,
      "Presidential Campaign": 0.82,
      "Republican": 0.82,
      "Debate": 0.78,
      "Politics": 0.99,
      "Government": 0.93,
      "Law": 0.85,
      "Elections": 0.72,
    },
    401: {
      "Trump": 0.95,
      "Felony": 0.92,
      "Business Records": 0.97,
      "Presidential Campaign": 0.84,
      "Republican": 0.82,
      "Former President": 0.98,
      "Conviction": 0.92,
      "Sentencing": 0.91,
      "Prison": 0.85,
      "14th Amendment": 0.82,
      "Insurrection": 0.80,
      "Supreme Court": 0.78,
      "Republican National Convention": 0.76,
      "Politics": 0.92,
      "Government": 0.92,
      "Law": 0.90,
      "Justice": 0.88,
      "Elections": 0.85,
    },
    51376: {
      "Immigration": 0.67,
      "Trump": 0.98,
      "Republican": 0.59,
      "College": 0.98,
      "Green Card": 0.93,
      "Legislation": 0.89,
      "Student": 0.89,
      "Politics": 0.82,
      "Government": 0.81,
      "Law": 0.69,
      "Education": 0.97
    }
  }
}
---
Now, the above was just an example. Now, do it for all the following text(s), generate many themes, make sure to return for each dataset ID!:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]
---
Remember, generate many themes and overarching themes, each one should be used in the classifications (don't skip any of the generated themes and overarching themes), then classify each dataset id by them, make sure to return for each dataset ID - [IDS] !
'''

In [ ]:
from llms import OpenAIGPTRepresentation

custom_response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "topic_analysis",
        "schema": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "object",
                    "properties": {
                        "themes": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of specific themes extracted from the texts"
                        },
                        "overarching_themes": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of high-level, general themes"
                        }
                    },
                    "required": ["themes", "overarching_themes"]
                },
                "texts": {
                    "type": "object",
                    "patternProperties": {
                        "^[0-9]+$": {
                            "type": "object",
                            "additionalProperties": {
                                "type": "number",
                                "minimum": 0,
                                "maximum": 1
                            }
                        }
                    }
                }
            },
            "required": ["topic", "texts"]
        }
    }
}

api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAIGPTRepresentation(api_key, model="gpt-4o-mini", prompt=prompt, generator_kwargs={"response_format": custom_response_format})

model.update_topics(data, topics=topics_reduced, representation_model=llm)

In [401]:
import json

# Directory containing the JSON files
cache_dir = './_cache/'
topic_list = model.get_topic_info()["Topic"].tolist()
openai_data = {}

for topic_id in topic_list:
    with open(f"{cache_dir}{topic_id}.json", 'r') as file:
        data = json.load(file)
        openai_data[topic_id] = data

# save as json
with open('topics_tags.json', 'w') as file:
    json.dump(openai_data, file, indent=4)

In [402]:
dataset_ids = augmented_df['dataset_id'].tolist()
print(len(dataset_ids))

# Collect all IDs from 'texts' keys in openai_data
found_ids = set()
for topic_id, topic_obj in openai_data.items():
    texts = topic_obj.get('texts', {})
    for key in texts.keys():
        try:
            found_ids.add(int(key))
        except ValueError:
            print(f"Skipping invalid key '{key}' in topic {topic_id}")

# Convert dataset_ids to a set of integers
dataset_ids_set = set(map(int, dataset_ids))

# Find missing IDs
missing_ids = dataset_ids_set - found_ids

# Print missing IDs
print("Missing dataset IDs:")
for missing_id in sorted(missing_ids):
    print(f"Dataset ID {missing_id} not found in any topic object")

5354
Missing dataset IDs:


In [403]:
dataset_id_to_tags = {}

for topic in openai_data.values():
    themes = topic.get('topic', {}).get('themes', [])
    overarching_themes = topic.get('topic', {}).get('overarching_themes', [])
    texts = topic.get('texts', {})
    for dataset_id_str, tags_scores in texts.items():
        try:
            dataset_id = int(dataset_id_str)
            if dataset_id not in dataset_id_to_tags:
                dataset_id_to_tags[dataset_id] = {'themes': {}, 'overarching_themes': {}}
            # Separate tags into themes and overarching_themes
            for tag, score in tags_scores.items():
                if tag == 'themes' and isinstance(score, dict):
                    # Process nested themes
                    for sub_tag, sub_score in score.items():
                        dataset_id_to_tags[dataset_id]['themes'][sub_tag] = sub_score
                    # Print dataset_id with nested 'themes'
                    print(f"Issue detected: Nested 'themes' in tags_scores for dataset_id {dataset_id}")
                elif tag == 'overarching_themes' and isinstance(score, dict):
                    # Process nested overarching_themes
                    for sub_tag, sub_score in score.items():
                        dataset_id_to_tags[dataset_id]['overarching_themes'][sub_tag] = sub_score
                    # Print dataset_id with nested 'overarching_themes'
                    print(f"Issue detected: Nested 'overarching_themes' in tags_scores for dataset_id {dataset_id}")
                else:
                    if tag in themes:
                        dataset_id_to_tags[dataset_id]['themes'][tag] = score
                    elif tag in overarching_themes:
                        dataset_id_to_tags[dataset_id]['overarching_themes'][tag] = score
                    else:
                        # Tag not found, placing in 'themes'
                        dataset_id_to_tags[dataset_id]['themes'][tag] = score
                        print(f"Tag '{tag}' not found in themes or overarching_themes for dataset_id {dataset_id}, placing in 'themes'")
        except ValueError:
            print(f"Skipping invalid dataset_id '{dataset_id_str}'")

# Save as JSON
with open('dataset_id_to_tags.json', 'w') as f:
    json.dump(dataset_id_to_tags, f, indent=4)

Tag 'Status' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CXC L9' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CXC L11' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CCL2' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'IFNG' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'PRF1' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CLU' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CCL3' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CXC L10' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'CD3E' not found in themes or overarching_themes for dataset_id 42799, placing in 'themes'
Tag 'GZMB' not found in themes or overarc